# Test Base Input Data

ほっといてもなかなかデータ出来ないのでとりあえず叩き台のインプットデータ作成

とりあえず
* ドル円為替（終値)
* 日経平均（終値)
* S&P500（終値, Volume, 修正終値）
* JGB10Y
* VIX（終値)
だけ用意.

またこれらに対して欠損日は線形補間とし、
* 絶対額
* 前日比
* 30日移動平均
* 120日移動平均
を暫定的に追加.

正規化はしていないので、インプット前に実施する必要あり.

## package読み込み

In [24]:
import quandl as ql
import pandas as pd

In [25]:
ql.ApiConfig.api_key = 'CbzNMfBFSFEFsiNNAe97' # APIKeyを置かないと取得制限

## 各種データ読み込み
後から追加可能

In [26]:
# 日本国債金利(10年)
df_Interest_JP = ql.get('MOFJ/INTEREST_RATE_JAPAN_10Y', start_date='2001-01-01')
df_Interest_JP.columns = ["JGB10"]
df_Interest_JP.head(10)

,JGB10
Date,
2001-01-04,1.639
2001-01-05,1.630
2001-01-09,1.565
2001-01-10,1.581
2001-01-11,1.519
2001-01-12,1.480
2001-01-15,1.499
2001-01-16,1.538
2001-01-17,1.553


In [27]:
# USD/JPY
df_JPYUSD = ql.get('FRED/DEXJPUS', start_date='2001-01-01')
df_JPYUSD.columns = ["JP/US"]
df_JPYUSD.head(10)

,JP/US
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97
2001-01-09,116.64
2001-01-10,116.26
2001-01-11,117.56
2001-01-12,117.76


In [28]:
# 日経平均株価
df_STOCKJP = ql.get('NIKKEI/INDEX'+".4", start_date='2001-01-01')
df_STOCKJP.columns = ["Nikkei/Close"]
df_STOCKJP.head(10)

,Nikkei/Close
Date,
2001-01-04,13691.49
2001-01-05,13867.61
2001-01-09,13610.51
2001-01-10,13432.65
2001-01-11,13201.07
2001-01-12,13347.74
2001-01-15,13506.23
2001-01-16,13584.45
2001-01-17,13667.63


In [29]:
# S&P株価　終値、ボリューム、修正後終値
df_STOCKSP = ql.get('YAHOO/INDEX_GSPC', start_date="2001-01-01")
df_STOCKSP = df_STOCKSP[["Close","Volume","Adjusted Close"]]
df_STOCKSP.columns = ["S&P/Close","S&P/Volume","S&P/AdjClose"]
df_STOCKSP.head(10)

,S&P/Close,S&P/Volume,S&P/AdjClose
Date,,,
2001-01-02,1283.270020,1.129400e+09,1283.270020
2001-01-03,1347.560059,1.880700e+09,1347.560059
2001-01-04,1333.339966,2.131000e+09,1333.339966
2001-01-05,1298.349976,1.430800e+09,1298.349976
2001-01-08,1295.859985,1.115500e+09,1295.859985
2001-01-09,1300.800049,1.191300e+09,1300.800049
2001-01-10,1313.270020,1.296500e+09,1313.270020
2001-01-11,1326.819946,1.411200e+09,1326.819946
2001-01-12,1318.550049,1.276000e+09,1318.550049


In [30]:
# VIX
df_VIX = ql.get('NIKKEI/INDEX'+".4", start_date='2001-01-01')
df_VIX.columns = ["VIX/Close"]
df_VIX.tail(10)

,VIX/Close
Date,
2016-06-24,14952.02
2016-06-27,15309.21
2016-06-28,15323.14
2016-06-29,15566.83
2016-06-30,15575.92
2016-07-01,15682.48
2016-07-04,15775.80
2016-07-05,15669.33
2016-07-06,15378.99


## Data前処理
* 結合
* 欠損処理
* 前日比データ追加
* 移動平均データ追加

In [31]:
#横方向への結合, 'inner'で存在する箇所のみ
merge_list = [df_JPYUSD, df_Interest_JP, df_STOCKJP, df_STOCKSP, df_VIX]
df_merge = pd.concat(merge_list, axis=1)
df_merge.tail(5)

,JP/US,JGB10,Nikkei/Close,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close
2016-07-01,102.55,-0.262,15682.48,2102.949951,3.458890e+09,2102.949951,15682.48
2016-07-04,NaN,-0.257,15775.80,NaN,NaN,NaN,15775.80
2016-07-05,NaN,-0.257,15669.33,2088.550049,3.658380e+09,2088.550049,15669.33
2016-07-06,NaN,-0.278,15378.99,2099.729980,3.909380e+09,2099.729980,15378.99
2016-07-07,NaN,NaN,15276.24,2097.899902,3.604550e+09,2097.899902,15276.24


In [32]:
# 欠損処理など
Data=df_merge.to_period(freq="D") # Dayをタイムピリオドに設定
Data = Data.interpolate() # データの前後で線形補間(データの前後以外nanはなくなる)
Data = Data.dropna(axis=0) # nanの行を落とす
Data.tail(5)

,JP/US,JGB10,Nikkei/Close,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close
2016-07-01,102.55,-0.262,15682.48,2102.949951,3.458890e+09,2102.949951,15682.48
2016-07-04,102.55,-0.257,15775.80,2095.750000,3.558635e+09,2095.750000,15775.80
2016-07-05,102.55,-0.257,15669.33,2088.550049,3.658380e+09,2088.550049,15669.33
2016-07-06,102.55,-0.278,15378.99,2099.729980,3.909380e+09,2099.729980,15378.99
2016-07-07,102.55,-0.278,15276.24,2097.899902,3.604550e+09,2097.899902,15276.24


In [33]:
# 前期比、移動平均
DataLag = Data.pct_change() # 前期比
DataLag.columns = [
    "JP/US.Lag", "JGB10.Lag", "Nikkei/Close.Lag", "S&P/Close.Lag" , "S&P/Volume.Lag", "S&P/AdjClose.Lag", "VIX/Close.Lag"]

DataMoveAve30 = pd.rolling_mean(Data,30) # 30day移動平均
DataMoveAve30.columns = [
    "JP/US.MA30", "JGB10.MA30", "Nikkei/Close.MA30", "S&P/Close.MA30" , "S&P/Volume.MA30", "S&P/AdjClose.MA30", "VIX/Close.MA30"]

DataMoveAve120 = pd.rolling_mean(Data,120) # 120day移動平均
DataMoveAve120.columns = [
    "JP/US.MA120", "JGB10.MA120", "Nikkei/Close.MA120", "S&P/Close.MA120" , "S&P/Volume.MA120", "S&P/AdjClose.MA120", "VIX/Close.MA120"]

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=30,center=False).mean()
/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=120,center=False).mean()


In [34]:
# 前期比と移動平均をマージ
InputList = [Data, DataLag, DataMoveAve30, DataMoveAve120 ]
InputData = pd.concat(InputList, axis=1)
InputData = InputData.dropna(axis=0) # nanの行を落とす
InputData.head(10)

,JP/US,JGB10,Nikkei/Close,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,Nikkei/Close.Lag,...,S&P/Volume.MA30,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,Nikkei/Close.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120
2001-06-20,123.30,1.201,12674.64,1223.140015,1.350100e+09,1223.140015,12674.64,0.002684,0.029135,0.007983,...,1.102100e+09,1261.507666,13461.619000,120.279083,1.321554,13386.141000,1254.843961,1.200070e+09,1254.843961,13386.141000
2001-06-21,124.40,1.182,12962.43,1237.040039,1.546820e+09,1237.040039,12962.43,0.008921,-0.015820,0.022706,...,1.118437e+09,1260.902999,13426.440333,120.353500,1.317746,13380.065500,1254.041461,1.195202e+09,1254.041461,13380.065500
2001-06-22,124.25,1.171,13044.61,1225.349976,1.189200e+09,1225.349976,13044.61,-0.001206,-0.009306,0.006340,...,1.127871e+09,1260.225663,13393.130000,120.420667,1.313921,13373.207167,1253.433128,1.193188e+09,1253.433128,13373.207167
2001-06-25,124.02,1.144,12896.47,1218.599976,1.050100e+09,1218.599976,12896.47,-0.001851,-0.023057,-0.011356,...,1.134267e+09,1259.214994,13360.578333,120.487750,1.310142,13366.185583,1252.789294,1.192644e+09,1252.789294,13366.185583
2001-06-26,123.82,1.140,12978.82,1216.760010,1.198900e+09,1216.760010,12978.82,-0.001613,-0.003497,0.006385,...,1.138504e+09,1258.125663,13324.738000,120.547583,1.306600,13360.921500,1252.088961,1.192707e+09,1252.088961,13360.921500
2001-06-27,124.25,1.133,12828.98,1211.069946,1.162100e+09,1211.069946,12828.98,0.003473,-0.006140,-0.011545,...,1.130397e+09,1255.661662,13295.895000,120.614167,1.302867,13355.890917,1251.237293,1.191587e+09,1251.237293,13355.890917
2001-06-28,124.63,1.125,12679.88,1226.199951,1.327300e+09,1226.199951,12679.88,0.003058,-0.007061,-0.011622,...,1.129454e+09,1253.585327,13254.868667,120.673083,1.299583,13351.547667,1250.398794,1.190888e+09,1250.398794,13351.547667
2001-06-29,124.73,1.183,12969.05,1224.380005,1.832360e+09,1224.380005,12969.05,0.000802,0.051556,0.022805,...,1.152839e+09,1251.332662,13224.578000,120.731167,1.297108,13348.391917,1249.614043,1.195524e+09,1249.614043,13348.391917
2001-07-02,124.22,1.263,12751.18,1236.719971,1.128300e+09,1236.719971,12751.18,-0.004089,0.067625,-0.016799,...,1.151286e+09,1248.795663,13177.056333,120.787000,1.295142,13342.099833,1248.898376,1.194586e+09,1248.898376,13342.099833
2001-07-03,124.48,1.272,12817.41,1234.449951,6.221100e+08,1234.449951,12817.41,0.002093,0.007126,0.005194,...,1.130010e+09,1246.297994,13134.597000,120.847000,1.292925,13335.707833,1248.130042,1.189723e+09,1248.130042,13335.707833


## InputデータとOutput(予測)データを分割

ここでは日経平均の騰落率の水準を予測

In [35]:
# 予測系列は除外
OutList = ["Nikkei/Close", "Nikkei/Close.Lag", "Nikkei/Close.MA30", "Nikkei/Close.MA120"]
InputDF = InputData.drop(OutList, axis=1)
InputDF.head(10)

,JP/US,JGB10,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,S&P/Close.Lag,S&P/Volume.Lag,...,S&P/Close.MA30,S&P/Volume.MA30,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120
2001-06-20,123.30,1.201,1223.140015,1.350100e+09,1223.140015,12674.64,0.002684,0.029135,0.008709,0.139421,...,1261.507666,1.102100e+09,1261.507666,13461.619000,120.279083,1.321554,1254.843961,1.200070e+09,1254.843961,13386.141000
2001-06-21,124.40,1.182,1237.040039,1.546820e+09,1237.040039,12962.43,0.008921,-0.015820,0.011364,0.145708,...,1260.902999,1.118437e+09,1260.902999,13426.440333,120.353500,1.317746,1254.041461,1.195202e+09,1254.041461,13380.065500
2001-06-22,124.25,1.171,1225.349976,1.189200e+09,1225.349976,13044.61,-0.001206,-0.009306,-0.009450,-0.231197,...,1260.225663,1.127871e+09,1260.225663,13393.130000,120.420667,1.313921,1253.433128,1.193188e+09,1253.433128,13373.207167
2001-06-25,124.02,1.144,1218.599976,1.050100e+09,1218.599976,12896.47,-0.001851,-0.023057,-0.005509,-0.116969,...,1259.214994,1.134267e+09,1259.214994,13360.578333,120.487750,1.310142,1252.789294,1.192644e+09,1252.789294,13366.185583
2001-06-26,123.82,1.140,1216.760010,1.198900e+09,1216.760010,12978.82,-0.001613,-0.003497,-0.001510,0.141701,...,1258.125663,1.138504e+09,1258.125663,13324.738000,120.547583,1.306600,1252.088961,1.192707e+09,1252.088961,13360.921500
2001-06-27,124.25,1.133,1211.069946,1.162100e+09,1211.069946,12828.98,0.003473,-0.006140,-0.004676,-0.030695,...,1255.661662,1.130397e+09,1255.661662,13295.895000,120.614167,1.302867,1251.237293,1.191587e+09,1251.237293,13355.890917
2001-06-28,124.63,1.125,1226.199951,1.327300e+09,1226.199951,12679.88,0.003058,-0.007061,0.012493,0.142156,...,1253.585327,1.129454e+09,1253.585327,13254.868667,120.673083,1.299583,1250.398794,1.190888e+09,1250.398794,13351.547667
2001-06-29,124.73,1.183,1224.380005,1.832360e+09,1224.380005,12969.05,0.000802,0.051556,-0.001484,0.380517,...,1251.332662,1.152839e+09,1251.332662,13224.578000,120.731167,1.297108,1249.614043,1.195524e+09,1249.614043,13348.391917
2001-07-02,124.22,1.263,1236.719971,1.128300e+09,1236.719971,12751.18,-0.004089,0.067625,0.010079,-0.384237,...,1248.795663,1.151286e+09,1248.795663,13177.056333,120.787000,1.295142,1248.898376,1.194586e+09,1248.898376,13342.099833
2001-07-03,124.48,1.272,1234.449951,6.221100e+08,1234.449951,12817.41,0.002093,0.007126,-0.001836,-0.448631,...,1246.297994,1.130010e+09,1246.297994,13134.597000,120.847000,1.292925,1248.130042,1.189723e+09,1248.130042,13335.707833


In [36]:
# Nikkeiを当てに行くことにする
OutputDF = InputData[OutList]
OutputDF.head(10)

,Nikkei/Close,Nikkei/Close.Lag,Nikkei/Close.MA30,Nikkei/Close.MA120
2001-06-20,12674.64,0.007983,13461.619000,13386.141000
2001-06-21,12962.43,0.022706,13426.440333,13380.065500
2001-06-22,13044.61,0.006340,13393.130000,13373.207167
2001-06-25,12896.47,-0.011356,13360.578333,13366.185583
2001-06-26,12978.82,0.006385,13324.738000,13360.921500
2001-06-27,12828.98,-0.011545,13295.895000,13355.890917
2001-06-28,12679.88,-0.011622,13254.868667,13351.547667
2001-06-29,12969.05,0.022805,13224.578000,13348.391917
2001-07-02,12751.18,-0.016799,13177.056333,13342.099833
2001-07-03,12817.41,0.005194,13134.597000,13335.707833


## 教師データの作成

次の日の騰落率を３区分に分類し、フラグを立てる

In [37]:
# 当てたいケースは次の日なのでずらす
OutputDF2 = OutputDF.join(OutputDF.shift(-1),rsuffix="_"+str(1))
OutputDF2.tail(10)

,Nikkei/Close,Nikkei/Close.Lag,Nikkei/Close.MA30,Nikkei/Close.MA120,Nikkei/Close_1,Nikkei/Close.Lag_1,Nikkei/Close.MA30_1,Nikkei/Close.MA120_1
2016-06-24,14952.02,-0.079216,16423.555000,16580.480375,15309.21,0.023889,16384.982000,16562.569958
2016-06-27,15309.21,0.023889,16384.982000,16562.569958,15323.14,0.000910,16340.660000,16546.771458
2016-06-28,15323.14,0.000910,16340.660000,16546.771458,15566.83,0.015903,16304.731333,16528.864792
2016-06-29,15566.83,0.015903,16304.731333,16528.864792,15575.92,0.000584,16269.040000,16514.989542
2016-06-30,15575.92,0.000584,16269.040000,16514.989542,15682.48,0.006841,16233.911000,16502.784292
2016-07-01,15682.48,0.006841,16233.911000,16502.784292,15775.80,0.005951,16204.617667,16492.952875
2016-07-04,15775.80,0.005951,16204.617667,16492.952875,15669.33,-0.006749,16176.970000,16481.460875
2016-07-05,15669.33,-0.006749,16176.970000,16481.460875,15378.99,-0.018529,16131.024667,16472.817542
2016-07-06,15378.99,-0.018529,16131.024667,16472.817542,15276.24,-0.006681,16081.150667,16466.642375
2016-07-07,15276.24,-0.006681,16081.150667,16466.642375,NaN,NaN,NaN,NaN


フラグを立てる時、for文ではなく内包表記など早い記法で書くこと

思いつかなかったのでそれぞれでフラグ立てて、足すことにした

In [38]:
# Closeの変化率を区切る
# 1%以上なら2,-1%以下なら1,それ以外は0
OutputAns = OutputDF2.assign(
    D = [2 if OutputDF2["Nikkei/Close.Lag_1"][x]  >= 0.01 else 0 for x in range(OutputDF2.shape[0])]).assign(
    E = [1 if OutputDF2["Nikkei/Close.Lag_1"][x]  <= -0.01 else 0 for x in range(OutputDF2.shape[0])]).assign(
    F = [0 if OutputDF2["Nikkei/Close.Lag_1"][x]  >= -0.01 and OutputDF2["Nikkei/Close.Lag_1"][x]  <= 0.01  else 0 for x in range(OutputDF2.shape[0])])[
    ["Nikkei/Close.Lag_1","D","E","F"]]
OutputAns.tail(10)

,Nikkei/Close.Lag_1,D,E,F
2016-06-24,0.023889,2,0,0
2016-06-27,0.000910,0,0,0
2016-06-28,0.015903,2,0,0
2016-06-29,0.000584,0,0,0
2016-06-30,0.006841,0,0,0
2016-07-01,0.005951,0,0,0
2016-07-04,-0.006749,0,0,0
2016-07-05,-0.018529,0,1,0
2016-07-06,-0.006681,0,0,0
2016-07-07,NaN,0,0,0


In [39]:
# 教師データを作成
OutputAns["Ans"] = OutputAns[["D","E","F"]].sum(axis=1)
OutputAns.tail(15)

,Nikkei/Close.Lag_1,D,E,F,Ans
2016-06-17,0.023439,2,0,0,2
2016-06-20,0.012766,2,0,0,2
2016-06-21,-0.006394,0,0,0,0
2016-06-22,0.010745,2,0,0,2
2016-06-23,-0.079216,0,1,0,1
2016-06-24,0.023889,2,0,0,2
2016-06-27,0.000910,0,0,0,0
2016-06-28,0.015903,2,0,0,2
2016-06-29,0.000584,0,0,0,0
2016-06-30,0.006841,0,0,0,0


## 学習用にInput Dataに教師データくっつける

別の学習用スクリプトの入力ファイルとする


In [40]:
InputCSV = pd.concat([InputDF,OutputAns], axis=1)
InputCSV = InputCSV.dropna(axis=0)
InputCSV = InputCSV.drop(["D","E","F"], axis=1)
InputCSV.tail(10)

,JP/US,JGB10,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,S&P/Close.Lag,S&P/Volume.Lag,...,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120,Nikkei/Close.Lag_1,Ans
2016-06-23,105.90,-0.146,2113.320068,3.297940e+09,2113.320068,16238.35,0.012816,-0.026667,0.013364,0.040964,...,2083.372172,16472.228000,111.618625,-0.035367,2012.685413,4.149947e+09,2012.685413,16603.363208,-0.079216,1
2016-06-24,102.26,-0.201,2037.300049,7.597450e+09,2037.300049,14952.02,-0.034372,0.376712,-0.035972,1.303696,...,2083.061841,16423.555000,111.489625,-0.038958,2013.645997,4.174385e+09,2013.645997,16580.480375,0.023889,2
2016-06-27,101.66,-0.202,2000.540039,5.431220e+09,2000.540039,15309.21,-0.005867,0.004975,-0.018043,-0.285126,...,2080.857845,16384.982000,111.357792,-0.042521,2014.286580,4.181251e+09,2014.286580,16562.569958,0.000910,0
2016-06-28,102.71,-0.233,2036.089966,4.385810e+09,2036.089966,15323.14,0.010329,0.153465,0.017770,-0.192482,...,2080.487179,16340.660000,111.232208,-0.046304,2015.098329,4.177072e+09,2015.098329,16546.771458,0.015903,2
2016-06-29,102.68,-0.242,2070.770020,4.241740e+09,2070.770020,15566.83,-0.000292,0.038627,0.017033,-0.032849,...,2081.258512,16304.731333,111.104042,-0.050071,2016.602412,4.170028e+09,2016.602412,16528.864792,0.000584,0
2016-06-30,102.77,-0.237,2098.860107,4.622820e+09,2098.860107,15575.92,0.000877,-0.020661,0.013565,0.089840,...,2083.219181,16269.040000,110.976875,-0.053971,2018.077580,4.164875e+09,2018.077580,16514.989542,0.006841,0
2016-07-01,102.55,-0.262,2102.949951,3.458890e+09,2102.949951,15682.48,-0.002141,0.105485,0.001949,-0.251779,...,2084.906844,16233.911000,110.858292,-0.057996,2019.932747,4.148129e+09,2019.932747,16502.784292,0.005951,0
2016-07-04,102.55,-0.257,2095.750000,3.558635e+09,2095.750000,15775.80,0.000000,-0.019084,-0.003424,0.028837,...,2086.497176,16204.617667,110.736042,-0.061854,2021.723747,4.134464e+09,2021.723747,16492.952875,-0.006749,0
2016-07-05,102.55,-0.257,2088.550049,3.658380e+09,2088.550049,15669.33,0.000000,0.000000,-0.003436,0.028029,...,2086.913509,16176.970000,110.610125,-0.065796,2023.450581,4.123881e+09,2023.450581,16481.460875,-0.018529,1
2016-07-06,102.55,-0.278,2099.729980,3.909380e+09,2099.729980,15378.99,0.000000,0.081712,0.005353,0.068610,...,2087.219840,16131.024667,110.494875,-0.069912,2025.453915,4.102992e+09,2025.453915,16472.817542,-0.006681,0


### CSV出力する場合は次をコメントアウトしてね

In [41]:
#InputCSV.to_csv("InputDNNMarket.csv")